### Load Packages

In [ ]:
import pandas as pd
import numpy as np
import os
import warnings
import datetime
import re

warnings.simplefilter("ignore")

pd.options.mode.chained_assignment = None

### Import Excel Files

In [ ]:
folder = r'C:\Users\mazhang\Documents\Jupyter Notebook\Monthly Closed Invoice\2022 Monthly Closed Invoice'
files = os.listdir(folder)

excel_list = []
for file in files:
    if file.endswith('.xlsx'):
        df = pd.read_excel(os.path.join(folder, file), sheet_name = None)
        excel_list.append(df)

### Generate Monthly Report with Accounting ID, Facility and  Bill Pay Category = Storage, etc.

In [ ]:
df_list = []

month = ['Jan', 'Feb', 'Mar']

for excel in range(len(excel_list)):
    
    df_header = excel_list[excel]['Header']
    
    df_detail = excel_list[excel]['Detail']
 

    df_select_detail_int = df_detail[['CustomerID', 'DocNo', 'WISE Facility',
                                      'Header Billing Period Start', 'Bill Pay Category']]
    df_select_detail = df_select_detail_int.loc[df_select_detail_int['Bill Pay Category'] == 'STORAGE INCOME']
    

    df_select_header = df_header[['Accounting_CustomerID', 'DocNo', 'Bill To']]
 

    df_combine_header_detail = pd.merge(df_select_detail, df_select_header, how = 'left',  on = ['DocNo']) 
    
    df_drop_dup_header = df_combine_header_detail.drop_duplicates()
    
    df_combine_rename = df_drop_dup_header.rename(columns = {'Header Billing Period Start':'Billing Period Start'})

    df_combine_month = df_combine_rename.reindex(columns = ['Accounting_CustomerID', 'CustomerID', 'WISE Facility', 
                                                            'Bill Pay Category', 'Bill To', 
                                                            'Billing Period Start', 'DocNo'])
    
    df_combine_month = df_combine_month.rename(columns = {'Billing Period Start':month[excel]})
    
    print(df_combine_month.shape)
    
    df_list.append(df_combine_month)

In [ ]:
df_list[len(df_list) - 1].sort_values('Accounting_CustomerID')

### Generate Monthly Report with Accounting ID, Facility and  Bill Pay Category = Storage, etc.

In [ ]:
df_combine_list = []
df_combine_quarter = df_list[0]

for df in range(1, len(df_list)):
    
    df_combine_quarter = pd.merge(df_combine_quarter, df_list[df], how = 'outer', 
                                  on = ['Accounting_CustomerID', 'CustomerID', 'WISE Facility', 'Bill Pay Category', 'Bill To'])
    print(df_combine_quarter.shape)
    
    df_combine_list.append(df_combine_quarter)
    
# df_output_init = df_combine_list[-1]
df_output_init = df_combine_quarter
df_output = df_output_init.drop_duplicates()

df_output.columns = df_output.columns.str.replace('_x', '')
df_output.columns = df_output.columns.str.replace('_y', '')

### Generate Monthly Report with Billing Count and without Invoice Number

In [ ]:
df_drop_iv = df_output.copy(deep = True)
df_drop_iv.drop('DocNo', inplace = True, axis = 1)

df_count = df_drop_iv.drop_duplicates()
df_count['Count'] = df_count.drop(df_count.columns[0:5], axis = 1).nunique(axis = 1)
df_count = df_count.sort_values(['WISE Facility', 'Accounting_CustomerID', 'CustomerID'])
df_count['WISE Facility'] = df_count['WISE Facility'].str.title()
df_count['Bill To'] = df_count['Bill To'].str.upper()
df_count = df_count.drop_duplicates(['WISE Facility', 'Accounting_CustomerID', 'CustomerID'])
df_count

In [ ]:
# df_count.to_excel("012022_012022 Monthly Report Storage Count.xlsx", index = False, header = True)

### Export Monthly Report As Excel File

In [ ]:
dn = np.where(df_output.columns.values == 'DocNo')

dn_month = [x + '_DocNo' for x in month]

df_output.columns.values[dn] = dn_month

In [ ]:
df_output = df_output.sort_values(['WISE Facility', 'Accounting_CustomerID', 'CustomerID'])
df_output['WISE Facility'] = df_output['WISE Facility'].str.title()
df_output['Bill To'] = df_output['Bill To'].str.upper()
df_output

In [ ]:
# df_output.to_excel("012022_012022 Monthly Report Storage.xlsx", index = False, header = True)

In [ ]:
writer1 = pd.ExcelWriter('2022 Monthly Report Storage.xlsx', engine = 'xlsxwriter')
df_count.to_excel(writer1, index = False, header = True, sheet_name = 'Summary')
df_output.to_excel(writer1, index = False, header = True, sheet_name = 'Detail 01-03')
writer1.save()

### Storage Cost

In [ ]:
df_list = []
month = ['Jan', 'Feb', 'Mar']

for excel in range(len(excel_list)):
    
    df_header = excel_list[excel]['Header']
    df_line = excel_list[excel]['Line']
    df_detail = excel_list[excel]['Detail']
 

    df_select_detail_int = df_detail[['CustomerID', 'DocNo', 'WISE Facility',
                                      'Header Billing Period Start', 'Bill Pay Category']]
    df_select_detail = df_select_detail_int.loc[df_select_detail_int['Bill Pay Category'] == 'STORAGE INCOME']
       
    df_select_header = df_header[['Accounting_CustomerID', 'DocNo', 'Bill To']]
 

    df_combine_header_detail = pd.merge(df_select_detail, df_select_header, how = 'left',  on = ['DocNo']) 
    
    df_drop_dup_header = df_combine_header_detail.drop_duplicates()
    
    df_combine_rename = df_drop_dup_header.rename(columns = {'Header Billing Period Start':'Billing Period Start'})

    df_combine_month = df_combine_rename.reindex(columns = ['Accounting_CustomerID', 'CustomerID', 'WISE Facility', 
                                                            'Bill Pay Category', 'Bill To', 'Billing Period Start',
                                                            'DocNo']).sort_values('Accounting_CustomerID')
    
    
    df_select_line_int = df_line[['CustomerID', 'WISE Facility', 'Bill Pay Category', 'Bill To', 
                                  'DocNo', 'Cost']].sort_values('CustomerID')
    df_select_line = df_select_line_int.loc[df_select_line_int['Bill Pay Category'] == 'STORAGE INCOME']
    
    
    
    df1 = df_select_line.groupby(['CustomerID', 'WISE Facility', 'Bill To'])['Cost'].sum().reset_index()
    
    df2 = df_combine_month.merge(df1, how = 'left', on = ['CustomerID', 'WISE Facility', 'Bill To'])
    
    df3 = df2.drop(['Billing Period Start', 'DocNo'], axis = 1)

    df3 = df3.rename(columns = {'Cost': month[excel]})
    
    print(df3.shape)
    
    df_list.append(df3)

In [ ]:
df_combine_list = []
df_combine_quarter = df_list[0]

for df in range(1, len(df_list)):
    
    df_combine_quarter = pd.merge(df_combine_quarter, df_list[df], how = 'outer', 
                                  on = ['Accounting_CustomerID', 'CustomerID', 'WISE Facility', 'Bill Pay Category', 'Bill To'])
    print(df_combine_quarter.shape)
    
    df_combine_list.append(df_combine_quarter)
    
# df_output_init = df_combine_list[-1]
df_output_init = df_combine_quarter
df_output = df_output_init.drop_duplicates()

In [ ]:
df_output

In [ ]:
df_drop_iv = df_output.copy(deep = True)

df_count = df_drop_iv.drop_duplicates()
df_count['Count'] = df_count.drop(df_count.columns[0:5], axis = 1).count(axis = 1)
df_count = df_count.sort_values(['WISE Facility', 'Accounting_CustomerID', 'CustomerID'])
df_count['WISE Facility'] = df_count['WISE Facility'].str.title()
df_count['Bill To'] = df_count['Bill To'].str.upper()

In [ ]:
df_count

In [ ]:
df_count.to_excel("2022 Monthly Report Storage Cost.xlsx", index = False, header = True)